# Creation of Vital Farms Donation Potential Dataset

## Imports and Reading in Sprouts and Whole Foods Location Data

In [2]:
# imports
import pandas as pd
import requests

In [17]:
# reading in Sprouts location data
sprouts_locs = pd.read_csv("sprouts_stores.csv")
sprouts_locs.head()

,State,Store Name,Street,City,Zip Code,Phone
0,AL,HOOVER,STORE #480 5250 Medford Dr. Suite #120,Hoover,35244.0,205-263-4970
1,AL,HUNTSVILLE – MADISON,STORE #481 7504 Hwy. 72 West,Madison,35758.0,256-203-8334
2,AL,BIRMINGHAM,STORE #482 Brook Highland Plaza 5265 US Hwy. 280,Birmingham,35242.0,205-263-2808
3,AZ,CHANDLER – RAY RD.,STORE #1 Boardwalk at Andersen Springs Center ...,Chandler,85224.0,480-732-1012
4,AZ,GILBERT-HIGLEY RD.,STORE #10 Higley Rd. & Baseline Rd. 1668 N. Hi...,Gilbert,85234.0,520-652-3524


In [18]:
# dropping Sprouts NA's since they are not useful in this case
sprouts_locs = sprouts_locs.dropna()
# casting Sprouts zip codes to integers for easy comparison
sprouts_locs['Zip Code'] = sprouts_locs['Zip Code'].astype(int)
sprouts_locs.head()

,State,Store Name,Street,City,Zip Code,Phone
0,AL,HOOVER,STORE #480 5250 Medford Dr. Suite #120,Hoover,35244,205-263-4970
1,AL,HUNTSVILLE – MADISON,STORE #481 7504 Hwy. 72 West,Madison,35758,256-203-8334
2,AL,BIRMINGHAM,STORE #482 Brook Highland Plaza 5265 US Hwy. 280,Birmingham,35242,205-263-2808
3,AZ,CHANDLER – RAY RD.,STORE #1 Boardwalk at Andersen Springs Center ...,Chandler,85224,480-732-1012
4,AZ,GILBERT-HIGLEY RD.,STORE #10 Higley Rd. & Baseline Rd. 1668 N. Hi...,Gilbert,85234,520-652-3524


In [19]:
# reading in Whole Foods location data
whole_foods_locs = pd.read_csv("whole_foods_stores.csv")
# dropping Whole Foods NA's since they are not useful in this case
whole_foods_locs = whole_foods_locs.dropna()
whole_foods_locs.head()

,Street Address,City,State,Zip Code
0,2501 Memorial Pkwy SW,Huntsville,AL,35801
1,3100 Cahaba Village Plaza,Birmingham,AL,35243
2,320 S Cambridge Ln,Flagstaff,AZ,86001
3,1420 W State Rte 89A,Sedona,AZ,86336
4,7111 E Mayo Blvd,Phoenix,AZ,85054


## Getting Socioeconomic Data from the American Community Survery (Census) API

In [21]:
# Defining the variables we want to retrieve from the Census ACS API
variables = [
    "B01003_001E",  # Total population
    "B19013_001E",  # Median household income
    "B17001_002E",  # Count of people in poverty
    "B23025_002E"   # Labor force participation
]

# Joining variables for URL
variable_str = ",".join(["NAME"] + variables)

# Building the URL for 2022 ACS for all zip codes
url = f"https://api.census.gov/data/2022/acs/acs5?get={variable_str}&for=zip%20code%20tabulation%20area:*"

print(f"Requesting data from: {url}")

# Make the API request
response = requests.get(url)

if response.status_code == 200:
    # Convert response to DataFrame
    data = response.json()
    headers = data[0]
    values = data[1:]
    
    # Create DataFrame
    acs_data = pd.DataFrame(values, columns=headers)
    
    # Convert numeric columns to proper data types
    for var in variables:
        if var != "NAME":
            acs_data[var] = pd.to_numeric(acs_data[var], errors='coerce')
    
    # Rename columns for better readability
    acs_data = acs_data.rename(columns={
        "B01003_001E": "total_population",
        "B19013_001E": "median_household_income",
        "B17001_002E": "poverty_count",
        "B23025_002E": "labor_force_count",
        "zip code tabulation area": "zcta"
    })
    
    # Calculate poverty rate
    acs_data["poverty_rate"] = acs_data["poverty_count"] / acs_data["total_population"] * 100
    
    # Display the first few rows
    print(acs_data.head())
    
else:
    print(f"Error: {response.status_code}")
    print(response.text)

Requesting data from: https://api.census.gov/data/2022/acs/acs5?get=NAME,B01003_001E,B19013_001E,B17001_002E,B23025_002E&for=zip%20code%20tabulation%20area:*
          NAME  total_population  median_household_income  poverty_count  \
0  ZCTA5 00601             16834                    17526          10440   
1  ZCTA5 00602             37642                    20260          17768   
2  ZCTA5 00603             49075                    17703          23551   
3  ZCTA5 00606              5590                    19603           3021   
4  ZCTA5 00610             25542                    22796          11597   

   labor_force_count   zcta  poverty_rate  
0               6058  00601     62.017346  
1              12182  00602     47.202593  
2              16136  00603     47.989812  
3               1449  00606     54.042934  
4               9542  00610     45.403649  


## Creating Master Dataset with ACS and Store Information

In [22]:
# Extract just the ZIP codes from store datasets
sprouts_zips = sprouts_locs['Zip Code'].astype(str).str.strip().tolist()
whole_foods_zips = whole_foods_locs['Zip Code'].astype(str).str.strip().tolist()

# Ensure ZIP codes in ACS data are strings for consistent comparison
acs_data['zcta'] = acs_data['zcta'].astype(str).str.strip()

# Create new columns for store presence based on zip codes
acs_data['has_sprouts'] = acs_data['zcta'].apply(lambda x: 'Yes' if x in sprouts_zips else 'No')
acs_data['has_whole_foods'] = acs_data['zcta'].apply(lambda x: 'Yes' if x in whole_foods_zips else 'No')

# Display the first few rows to verify
acs_data.head()

,NAME,total_population,median_household_income,poverty_count,labor_force_count,zcta,poverty_rate,has_sprouts,has_whole_foods
0,ZCTA5 00601,16834,17526,10440,6058,00601,62.017346,No,No
1,ZCTA5 00602,37642,20260,17768,12182,00602,47.202593,No,No
2,ZCTA5 00603,49075,17703,23551,16136,00603,47.989812,No,No
3,ZCTA5 00606,5590,19603,3021,1449,00606,54.042934,No,No
4,ZCTA5 00610,25542,22796,11597,9542,00610,45.403649,No,No


In [23]:
# Save the dataset to be used for the visualizations
acs_data.to_csv("acs_data_with_store_presence.csv", index=False)